<a href="https://colab.research.google.com/github/Felipehonorato1/NLPcourse/blob/main/Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  !pip install -q tf-nightly
except Exception:
  pass
import tensorflow as tf

     |████████████████████████████████| 410.0MB 29kB/s 
     |████████████████████████████████| 471kB 29.3MB/s 
     |████████████████████████████████| 4.0MB 20.9MB/s 
     |████████████████████████████████| 4.0MB 29.9MB/s 
     |████████████████████████████████| 6.1MB 30.1MB/s 
     |████████████████████████████████| 3.8MB 47.5MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.


In [2]:
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [3]:
embedding_layer = layers.Embedding(1000,5)

In [4]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[-0.00625554, -0.04516646, -0.01447616,  0.01226547,  0.02309803],
       [-0.00484567,  0.03040225,  0.02234891, -0.03812521,  0.02750755],
       [ 0.03580947,  0.02987064, -0.0461934 , -0.03907194,  0.02659646]],
      dtype=float32)

In [5]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

In [6]:
result.numpy()

array([[[-0.00830698,  0.036154  , -0.01115374,  0.02792274,
          0.02079723],
        [-0.00625554, -0.04516646, -0.01447616,  0.01226547,
          0.02309803],
        [-0.00484567,  0.03040225,  0.02234891, -0.03812521,
          0.02750755]],

       [[ 0.03580947,  0.02987064, -0.0461934 , -0.03907194,
          0.02659646],
        [-0.04760091,  0.04617335, -0.02323394, -0.03621348,
         -0.02690537],
        [ 0.02526644, -0.02448936, -0.02179408,  0.02349227,
         -0.02993479]]], dtype=float32)

In [7]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteYL09XR/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteYL09XR/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteYL09XR/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [15]:
encoder = info.features['text'].encoder

In [8]:
train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

In [14]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy().shape

(10, 562)

In [16]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Epoch 1/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.6230 - accuracy: 0.6603 - val_loss: 0.4315 - val_accuracy: 0.8500
Epoch 2/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.3065 - accuracy: 0.8871 - val_loss: 0.2996 - val_accuracy: 0.9000
Epoch 3/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.2403 - accuracy: 0.9114 - val_loss: 0.4155 - val_accuracy: 0.8600
Epoch 4/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.2071 - accuracy: 0.9256 - val_loss: 0.4157 - val_accuracy: 0.8650
Epoch 5/10
2500/2500 [==============================] - 13s 5ms/step - loss: 0.1828 - accuracy: 0.9374 - val_loss: 0.3531 - val_accuracy: 0.8850
Epoch 6/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.1606 - accuracy: 0.9442 - val_loss: 0.4112 - val_accuracy: 0.8700
Epoch 7/10
2500/2500 [==============================] - 12s 5ms/step - loss: 0.1467 - accuracy: 0.9499 - val_loss: 0.5646 - val_ac